# Population LFP from quantitative model

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from brainsignals.plotting_convention import mark_subplots, simplify_axes

fig = plt.figure(figsize=[6, 3.5])
fig.subplots_adjust(bottom=0.1, top=0.92, right=0.97, wspace=0.2)
ax1 = fig.add_subplot(221, xlabel="$R$ (µm)", ylabel=r"$V_{{\rm e}\sigma}$ (µV)", ylim=[0, 200], xlim=[0, 1000])
ax2 = fig.add_subplot(222, xlabel="$R$ (µm)", ylabel=r"$V_{{\rm e}\sigma}$ (µV)", ylim=[0, 10], xlim=[0, 1000])
ax3 = fig.add_subplot(223, xlabel="$R$ (µm)", ylabel=r"$V_{{\rm e}\sigma}$ (µV)", ylim=[0, 2000])
ax4 = fig.add_subplot(224, xlabel="$R$ (µm)", ylabel=r"$V_{{\rm e}\sigma}$ (µV)", ylim=[0, 800])
    
# In soma region, uncorrelated
F_0 = 10  # µV
R = np.linspace(0, 2000, 2000)
r_eps = 10  # µm
r_star = 60  # µm

rho = 5000 * 1e-6 # µm⁻²

Ve_su = np.zeros(len(R))
Ve_su_ = np.zeros(len(R))
Ve_su[R < r_eps] = F_0 * np.sqrt(rho * np.pi) * R[R < r_eps]
Ve_su[(r_eps <= R) & (R < r_star)] = F_0 * np.sqrt(rho * np.pi) * np.sqrt(
    r_eps * (2 * R[(r_eps <= R) & (R < r_star)] - r_eps))
Ve_su[R >= r_star] = F_0 * np.sqrt(rho * np.pi) * np.sqrt(
    r_eps * (3 * r_star - r_eps - r_star**3 / R[R >= r_star]**2))

Ve_su_[R < r_eps] = F_0 * np.sqrt(rho * np.pi) * R[R < r_eps]
Ve_su_[(r_eps <= R)] = F_0 * np.sqrt(rho * np.pi) * np.sqrt(
    r_eps * (2 * R[(r_eps <= R)] - r_eps))

Ve_su_bigR = F_0 * np.sqrt(rho * np.pi) * np.sqrt(r_eps * (3 * r_star - r_eps))

ax1.plot(R, Ve_su, c='k')
ax1.plot(R, Ve_su_, c='gray', ls='--')
#ax1.axvline(r_star, ls='--', c='gray')
ax1.axhline(Ve_su_bigR, ls=':', c='gray')

# Not in soma region, uncorrelated
r_star = 300  # µm
F_0 = 0.1  # µV
Ve_au = np.zeros(len(R))
Ve_au_ = np.zeros(len(R))
Ve_au[R <= r_star] = F_0 * np.sqrt(rho * np.pi) * R[R <= r_star]
Ve_au[r_star < R]  = F_0 * np.sqrt(rho * np.pi) * r_star * np.sqrt(
    (2 - r_star**2 / R[R > r_star]**2))

Ve_au_ = F_0 * np.sqrt(rho * np.pi) * R
Ve_au_bigR = F_0 * np.sqrt(2 * rho * np.pi) * r_star

ax2.plot(R, Ve_au, c='k')
ax2.plot(R, Ve_au_, c='gray', ls='--')
ax2.axhline(Ve_au_bigR, ls=':', c='gray')


# In soma region, correlated
F_0 = 10
r_eps = 10
r_star = 60

Ve_sc = np.zeros(len(R))
Ve_sc_ = np.zeros(len(R))
Ve_sc[R < r_eps] = F_0 * rho * np.pi * R[R < r_eps]**2
Ve_sc[(r_eps <= R) & (R < r_star)] = F_0 * rho * np.pi / 3 * (
    4 * np.sqrt(r_eps) * R[(r_eps <= R) & (R < r_star)]**(3/2) - r_eps**2)
Ve_sc[R >= r_star] = F_0 * rho * np.pi / 3 * ((6 * np.log(R[R >= r_star] / r_star) + 4) * np.sqrt(r_eps) 
    * r_star **(3/2) -r_eps**2)

Ve_s = lambda c: np.sqrt((1-c) * Ve_su**2 + c * Ve_sc**2)

l1, = ax3.plot(R, Ve_s(1), c='k', ls='-')
l2, = ax3.plot(R, Ve_s(0.1), c='k', ls='--')
l3, = ax3.plot(R, Ve_s(0.01), c='k', ls=':')


# Not in soma region, correlated
r_star = 300
F_0 = 0.1
Ve_ac = np.zeros(len(R))
Ve_ac_ = np.zeros(len(R))
Ve_ac[R <= r_star] = F_0 * rho * np.pi * R[R <= r_star]**2
Ve_ac[r_star < R]  = F_0 * rho * np.pi * r_star**2 * (2 * np.log(R[R > r_star] / r_star) + 1)

Ve_a = lambda c: np.sqrt((1-c) * Ve_au ** 2 + c * Ve_ac**2)


ax4.plot(R, Ve_a(1), c='k', ls='-')
ax4.plot(R, Ve_a(0.1), c='k', ls='--')
ax4.plot(R, Ve_a(0.01), c='k', ls=':')

ax3.legend([l1, l2, l3], [r"$c_{\rm Ve}$=1", 
                          r"$c_{\rm Ve}$=0.1", 
                          r"$c_{\rm Ve}$=0.01"], frameon=False, ncol=1, loc=(0.02, 0.69))


ax4.legend([l1, l2, l3], [r"$c_{\rm Ve}$=1", 
                          r"$c_{\rm Ve}$=0.1", 
                          r"$c_{\rm Ve}$=0.01"], frameon=False, ncol=1, loc=(0.02, 0.69))

simplify_axes(fig.axes)
mark_subplots(fig.axes)
plt.savefig("shape_function_pop_phenomenological.pdf")

In [ ]:
from scipy.integrate import dblquad

def F_func(r, r_star, r_eps, F_0):
    if r < r_eps:
        return F_0
    elif r_eps <= r < r_star:
        return F_0 * np.sqrt(r_eps / r)
    elif r > r_star:
        return F_0 * np.sqrt(r_eps / r_star) * (r_star / r) **2

def integrand_u(theta, r_, r, r_star, r_eps, F_0):
    r__ = np.sqrt((r - r_ * np.cos(theta))**2 + (r_ * np.sin(theta))**2)
    return r_ * F_func(r__, r_star, r_eps, F_0)**2

def integrand_c(theta, r_, r, r_star, r_eps, F_0):
    r__ = np.sqrt((r - r_ * np.cos(theta))**2 + (r_ * np.sin(theta))**2)
    return r_ * F_func(r__, r_star, r_eps, F_0)


#r_star = 60.
r_eps = 10.
F_0 = 10.
R_ = 500
rs = np.linspace(0, 1500, 300)
Ve_su = np.zeros(len(rs))
Ve_sc = np.zeros(len(rs))
Ve_au = np.zeros(len(rs))
Ve_ac = np.zeros(len(rs))


for idx, r in enumerate(rs):
    r_star = 60
    Ve_su[idx] = np.sqrt(rho * dblquad(integrand_u, 0, R_, 0, 2 * np.pi, args=(r, r_star, r_eps, F_0))[0])
    Ve_sc[idx] = rho * dblquad(integrand_c, 0, R_, 0, 2 * np.pi, args=(r, r_star, r_eps, F_0))[0]
    
    r_star = 200
    Ve_au[idx] = np.sqrt(rho * dblquad(integrand_u, 0, R_, 0, 2 * np.pi, args=(r, r_star, r_eps, F_0))[0])
    Ve_ac[idx] = rho * dblquad(integrand_c, 0, R_, 0, 2 * np.pi, args=(r, r_star, r_eps, F_0))[0]
    
    print(r, Ve_su[idx], Ve_sc[idx])
    
Ve_s = lambda c: np.sqrt((1 - c) * Ve_su**2 + c * Ve_sc**2) 
Ve_a = lambda c: np.sqrt((1 - c) * Ve_au**2 + c * Ve_ac**2) 


Ve_s_500_1 = Ve_s(0)
Ve_s_500_2 = Ve_s(0.1)
Ve_s_500_3 = Ve_s(1)

Ve_a_500_1 = Ve_a(0)
Ve_a_500_2 = Ve_a(0.1)
Ve_a_500_3 = Ve_a(1)

r_star = 60

R_ = 200
#rs = np.linspace(0, 1500, 30)
Ve_su = np.zeros(len(rs))
Ve_sc = np.zeros(len(rs))
for idx, r in enumerate(rs):
    Ve_su[idx] = np.sqrt(rho * dblquad(integrand_u, 0, R_, 0, 2 * np.pi, args=(r, r_star, r_eps, F_0))[0])
    Ve_sc[idx] = rho * dblquad(integrand_c, 0, R_, 0, 2 * np.pi, args=(r, r_star, r_eps, F_0))[0]
    print(r, Ve_su[idx], Ve_sc[idx])

Ve_s_200 = lambda c: np.sqrt((1 - c) * Ve_su**2 + c * Ve_sc**2) 
Ve_s_200_2 = Ve_s_200(0.1)
    
R_ = 100
#rs = np.linspace(0, 3000, 300)
Ve_su = np.zeros(len(rs))
Ve_sc = np.zeros(len(rs))
for idx, r in enumerate(rs):
    Ve_su[idx] = np.sqrt(rho * dblquad(integrand_u, 0, R_, 0, 2 * np.pi, args=(r, r_star, r_eps, F_0))[0])
    Ve_sc[idx] = rho * dblquad(integrand_c, 0, R_, 0, 2 * np.pi, args=(r, r_star, r_eps, F_0))[0]
    print(r, Ve_su[idx], Ve_sc[idx])
Ve_s_100 = lambda c: np.sqrt((1 - c) * Ve_su**2 + c * Ve_sc**2) 
Ve_s_100_2 = Ve_s_100(0.1)


In [ ]:
fig = plt.figure()
fig.subplots_adjust(right=0.96, top=0.9, left=0.09, wspace=0.25, bottom=0.13)
ax1 = fig.add_subplot(221, xlabel="$r$ (µm)", ylabel=r"$V_{{\rm e}\sigma}$ (µV)", ylim=[1e-1, 2e3], yscale="log")
ax2 = fig.add_subplot(222, xlim=[0, 3], ylim=[0.5e1, 1e3], xlabel="$r$ / $R$", ylabel=r"$V_{{\rm e}\sigma}$ (µV)")
ax3 = fig.add_subplot(223, xlim=[0, 1500], ylim=[0, 1.05], xlabel="$r$ (µm)", ylabel=r"norm. $V_{{\rm e}\sigma}$")
ax4 = fig.add_subplot(224, xlim=[0, 1500], ylim=[0., 1.05], xlabel="$r$ (µm)", ylabel=r"norm. $V_{{\rm e}\sigma}$")


l1, = ax1.semilogy(rs, Ve_s_500_1, 'r')
l2, = ax1.semilogy(rs, Ve_s_500_2, 'b')
l3, = ax1.semilogy(rs, Ve_s_500_3, 'k')

ax1.axvline(500, c='gray', ls='--', lw=1)
ax2.axvline(1, c='gray', ls='--', lw=1)
ax3.axvline(500, c='gray', ls='--', lw=1)
ax4.axvline(500, c='gray', ls='--', lw=1)

ax1.legend([l3, l2, l1], [r"$c_{\rm Ve}$=1", r"$c_{\rm Ve}$=0.1", r"$c_{\rm Ve}$=0"], 
           frameon=False, ncol=1, loc=(0.7, 0.6))

l1, = ax2.semilogy(rs / 500, Ve_s_500_2, 'k')
l2, = ax2.semilogy(rs / 200, Ve_s_200_2, 'b')
l3, = ax2.semilogy(rs / 100, Ve_s_100_2, 'r')

ax2.legend([l1, l2, l3], ["$R$=500 µm", "$R$=200 µm", "$R$=100 µm"], 
           frameon=False, ncol=1, loc=(0.5, 0.6))

l1, = ax3.plot(rs, Ve_s_500_1 / np.max(Ve_s_500_1), c='k')
l2, = ax3.plot(rs, Ve_a_500_1 / np.max(Ve_a_500_1), c='r')
ax3.legend([l1, l2], [r"$c_{\rm Ve}$=0; soma", r"$c_{\rm Ve}$=0; apical"], 
           frameon=False, ncol=1, loc=(0.5, 0.5))

l1, = ax4.plot(rs, Ve_s_500_3 / np.max(Ve_s_500_3), c='k')
l2, = ax4.plot(rs, Ve_a_500_3 / np.max(Ve_a_500_3), c='r')
ax4.legend([l1, l2], [r"$c_{\rm Ve}$=1; soma", r"$c_{\rm Ve}$=1; apical"], 
           frameon=False, ncol=1, loc=(0.5, 0.5))
import matplotlib.ticker

simplify_axes([ax1, ax2, ax3, ax4])
mark_subplots([ax1, ax2, ax3, ax4])
ax1.set_yticks([1e-1, 1e0, 1e1, 1e2, 1e3])

ax1.yaxis.set_minor_locator(matplotlib.ticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))
fig.savefig("decay_outside_pop.pdf")